In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import re
import matplotlib.pyplot as plt
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
y= pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
y.head()

In [ ]:
print(y.breed.unique())
print("\n")
print(y.breed.nunique())
print("\n")
print(y.isna().sum())
print("\n")
print(y.breed.value_counts())
print("\n")

img=cv2.imread('../input/dog-breed-identification/test/ddc0134def0da0472e55271da8b57c44.jpg')
#('../input/dog-breed-identification/test/000621fb3cbb32d8935728e48679680e.jpg')

img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()



In [ ]:
print(img.shape)
H=128
W=128
C=3

train_file_location = '../input/dog-breed-identification/train/' 
train_data = y.assign(img_path = lambda x : train_file_location + x['id'] + '.jpg')
train_data.head()



In [ ]:
from keras.preprocessing.image import load_img,img_to_array, ImageDataGenerator
X = np.array([img_to_array(load_img(img,target_size = (H,W))) for img in train_data['img_path'].values.tolist()])
print(X.shape)
Y = pd.get_dummies(train_data['breed'])
print(Y.shape)


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test  = train_test_split(X,Y,test_size = 0.25)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten,Activation
model=Sequential()

model.add(Conv2D(32,(3,3),input_shape=(H,W,C)))
model.add(Activation('relu'))

model.add(MaxPool2D((2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))

model.add(MaxPool2D((2,2)))

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))

model.add(MaxPool2D((2,2)))

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))

model.add(MaxPool2D((2,2)))

model.add(Flatten())

model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(Y.shape[1]))
model.add(Activation('softmax'))

model.summary()



In [ ]:
batch=32

model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy'])

trained_model=model.fit(X_train,Y_train,
         epochs=20,
         batch_size=batch,
         steps_per_epoch=X_train.shape[0]//batch,
         validation_steps=X_test.shape[0]//batch,
         validation_data=(X_test,Y_test),
         verbose=2)



In [ ]:
test_datagen = ImageDataGenerator()

test_set = test_datagen.flow_from_directory(
    '/kaggle/input/dog-breed-identification',
    target_size = (128,128),
    classes=['test']
)
y_pred = model.predict(test_set)

In [ ]:
submission = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')
submission.head()

In [ ]:

file_list = test_set.filenames
id_list = []
for name in file_list:
    m = re.sub('test/', '', name)
    m = re.sub('.jpg', '', m)
    id_list.append(m)
    
submission['id'] = id_list
submission.iloc[:,1:] =y_pred
submission.head()

In [ ]:
r = submission.set_index('id')
r.to_csv('submission.csv')